In [ ]:
pip install wget

In [1]:
import astropy
from astropy.table import Table,vstack, Column
from subprocess import call
import subprocess
from astropy.io import fits
import numpy as np
import wget
import matplotlib.pyplot as plt
from gpz_pype.utilities import Params, set_gpz_path, basic_lupt_soft, flux_to_lupt
import astropy.units as u
from numpy.ma import masked
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky

In [ ]:
fields= ['ceers-full','gdn','gds','ngdeep','primer-cosmos-east','primer-cosmos-west','primer-uds-north','primer-uds-south','abell2744clu','abells1063','elgordo','j112716p4228','macs0647','rxj2129','smacs0723','sunrise']

# Download Field Files


In [ ]:
for field in fields:
    url=f'https://s3.amazonaws.com/grizli-v2/JwstMosaics/v7/{field}-grizli-v7.0_phot.fits'
    filename=wget.download(url)
    print(f' {field} downloaded')

# Initial Catalogue Making - Run Once


In [ ]:
###INITIAL CATALOGUE MAKING
hdu_list = fits.open('MPhys/DAWN_JWST/ceers-full-grizli-v7.0_phot.fits', memmap=True,formats='fits')
ceers = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/gdn-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gdn= Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/gds-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gds = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/ngdeep-grizli-v7.0_phot.fits', memmap=True,formats='fits')
ngdeep = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-cosmos-east-grizli-v7.0_phot.fits', memmap=True,formats='fits')
cosmos_east = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-cosmos-west-grizli-v7.0_phot.fits', memmap=True,formats='fits')
cosmos_west = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-uds-south-grizli-v7.0_phot.fits', memmap=True,formats='fits')
udss = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-uds-north-grizli-v7.0_phot.fits', memmap=True,formats='fits')
udsn = Table(hdu_list[1].data)
hdu_list.close()

In [ ]:
#ADDING FIELDS TO CATALOGUE - RUN BOTH CELLS IF STARTING FROM SCRATCH
hdu_list = fits.open('MPhys/DAWN_JWST/abell2744clu-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abell2744 = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/abells1063-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abells1063= Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/elgordo-grizli-v7.0_phot.fits', memmap=True,formats='fits')
elgordo = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/j112716p4228-grizli-v7.0_phot.fits', memmap=True,formats='fits')
j_field = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/macs0647-grizli-v7.0_phot.fits', memmap=True,formats='fits')
macs = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/rxj2129-grizli-v7.0_phot.fits', memmap=True,formats='fits')
rxj = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/smacs0723-grizli-v7.0_phot.fits', memmap=True,formats='fits')
smacs = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/sunrise-grizli-v7.0_phot.fits', memmap=True,formats='fits')
sunrise = Table(hdu_list[1].data)
hdu_list.close()

In [ ]:
abell2744.add_column('abells2744clu', name='field', index=0)
abells1063.add_column('abells1063', name='field', index=0)
elgordo.add_column('elgordo', name='field', index=0)
j_field.add_column('j112716p4228', name='field', index=0)
macs.add_column('macs0647', name='field', index=0)
rxj.add_column('rxj2129', name='field', index=0)
smacs.add_column('smacs0723', name='field', index=0)
sunrise.add_column('sunrise', name='field', index=0)

In [ ]:
ceers.add_column('ceers', name='field', index=0)
gdn.add_column('gdn', name='field', index=0)
gds.add_column('gds', name='field', index=0)
ngdeep.add_column('ngdeep', name='field', index=0)
cosmos_east.add_column('cosmos_east', name='field', index=0)
cosmos_west.add_column('cosmos_west', name='field', index=0)
udsn.add_column('udsn', name='field', index=0)
udss.add_column('udss', name='field', index=0)

In [ ]:
fields_new= ['ceers','gdn','gds','ngdeep','cosmos_east','cosmos_west','udsn','udss','abell2744','abells1063','elgordo','j_field','macs','rxj','smacs','sunrise']

# Convert to Luptiudes for Each Field


In [ ]:
field=sunrise


In [ ]:
filters0 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_0')]
filters0=filters0[1:]
filters1 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_1')]
filters1=filters1[1:]

filters2 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_2')]
filters2=filters2[1:]

filters3 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_3')]
filters3=filters3[1:]



In [ ]:

b_arr0 = [basic_lupt_soft(field[f'{filt}_flux_aper_0'], field[f'{filt}_fluxerr_aper_0']) for filt in filters0] 
b_arr1 = [basic_lupt_soft(field[f'{filt}_flux_aper_1'], field[f'{filt}_fluxerr_aper_1']) for filt in filters1] 
b_arr2 = [basic_lupt_soft(field[f'{filt}_flux_aper_2'], field[f'{filt}_fluxerr_aper_2']) for filt in filters2] 
b_arr3 = [basic_lupt_soft(field[f'{filt}_flux_aper_3'], field[f'{filt}_fluxerr_aper_3']) for filt in filters3] 

In [ ]:
i=15
lupt_cols=field['field','id','ra','dec']

for filt,b in zip(filters0,b_arr0):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_0'],field[f'{filt}_fluxerr_aper_0'],b)
    lupt_cols[f'lupt_{filt}_aper0']=lupt
    lupt_cols[f'lupterr_{filt}_aper0']=lupterr
for filt,b in zip(filters1,b_arr1):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_1'],field[f'{filt}_fluxerr_aper_1'],b)
    lupt_cols[f'lupt_{filt}_aper1']=lupt
    lupt_cols[f'lupterr_{filt}_aper1']=lupterr
for filt,b in zip(filters2,b_arr2):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_2'],field[f'{filt}_fluxerr_aper_2'],b)
    lupt_cols[f'lupt_{filt}_aper2']=lupt
    lupt_cols[f'lupterr_{filt}_aper2']=lupterr
for filt,b in zip(filters3,b_arr3):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_3'],field[f'{filt}_fluxerr_aper_3'],b)
    lupt_cols[f'lupt_{filt}_aper3']=lupt
    lupt_cols[f'lupterr_{filt}_aper3']=lupterr

lupt_cols.write(f'MPhys/catalogues/{fields[i]}_lupts.fits',format='fits', overwrite=True)

In [2]:
lup_ceers=Table.read('MPhys/catalogues/ceers-full_lupts.fits',format='fits')
lup_gds=Table.read('MPhys/catalogues/gds_lupts.fits',format='fits')
lup_gdn=Table.read('MPhys/catalogues/gdn_lupts.fits',format='fits')
lup_ngdeep=Table.read('MPhys/catalogues/ngdeep_lupts.fits',format='fits')
lup_udss=Table.read('MPhys/catalogues/primer-uds-south_lupts.fits',format='fits')
lup_udsn=Table.read('MPhys/catalogues/primer-uds-north_lupts.fits',format='fits')
lup_cosmos_east=Table.read('MPhys/catalogues/primer-cosmos-east_lupts.fits',format='fits')
lup_cosmos_west=Table.read('MPhys/catalogues/primer-cosmos-west_lupts.fits',format='fits')

In [3]:
lup_abell2744clu = Table.read('MPhys/catalogues/abell2744clu_lupts.fits',format='fits')
lup_abells1063=Table.read('MPhys/catalogues/abells1063_lupts.fits',format='fits')
lup_elgordo=Table.read('MPhys/catalogues/elgordo_lupts.fits',format='fits')
lup_j112716p4228=Table.read('MPhys/catalogues/j112716p4228_lupts.fits',format='fits')
lup_macs0647=Table.read('MPhys/catalogues/macs0647_lupts.fits',format='fits')
lup_rxj2129=Table.read('MPhys/catalogues/rxj2129_lupts.fits',format='fits')
lup_smacs0723=Table.read('MPhys/catalogues/smacs0723_lupts.fits',format='fits')
lup_sunrise=Table.read('MPhys/catalogues/sunrise_lupts.fits',format='fits')

In [4]:
#saves laptop dying if split up 
lup_con =vstack([lup_ceers,lup_gds,lup_gdn,lup_ngdeep,lup_udss,lup_udsn,lup_cosmos_east,lup_cosmos_west,])


In [5]:
lup_con_small = vstack([lup_abell2744clu,lup_abells1063,lup_elgordo,lup_j112716p4228,lup_macs0647,lup_rxj2129,lup_smacs0723,lup_sunrise])

In [6]:
lup_total = vstack([lup_con,lup_con_small])

In [7]:
#needs to be done in CP Lab 
lup_total.write('luptitudes_all_jwst.fits', format='fits', overwrite=True)


# Skymatching Catalogues

In [2]:
candels=Table.read('MPhys/CANDELS_dawn_catalogue.fits',format='fits')
jwst=Table.read('luptitudes_all_jwst.fits',format='fits')

In [4]:
ra1=jwst['ra']
dec1=jwst['dec']
ra2=candels['RA']
dec2=candels['DEC']

In [5]:
c = SkyCoord(ra=ra1*u.arcsec, dec=dec1*u.arcsec)#jwst
catalog = SkyCoord(ra=ra2*u.arcsec, dec=dec2*u.arcsec)#candels

In [6]:
max_sep = 1.0 * u.arcsec
idx, d2d, d3d = c.match_to_catalog_sky(catalog) #catalog = candels information
sep_constraint = d2d < max_sep
#c_matches = c[sep_constraint]
#catalog_matches = catalog[idx[sep_constraint]]

In [8]:
tst=[] #ids of candels catalog 
for i in range(len(ra1)):
    if d2d[i]< max_sep:
        tst.append(idx[i])
    
    

In [20]:
idx_new = list(set(tst)) #list of non-repeating candels indexs
matched_candels = candels[idx_new]
matched_candels.sort('RA')
matched_candels

ID,RA,DEC,CLASS_STAR,FLAGS,EBV,z_spec
,,,"0 galaxy, 1 star","0 use, gt 1 no-use",,
int64,float64,float64,float64,int64,float64,float64
-9223372036854775808,3.564901,-30.42496,--,-9223372036854775808,--,5.331
-9223372036854775808,3.5652,-30.39426,--,-9223372036854775808,--,5.772
-9223372036854775808,3.570643,-30.41464,--,-9223372036854775808,--,5.763
-9223372036854775808,3.577001,-30.41552,--,-9223372036854775808,--,5.133
-9223372036854775808,3.577166,-30.42258,--,-9223372036854775808,--,4.584
-9223372036854775808,3.580275,-30.4244,--,-9223372036854775808,--,4.015
-9223372036854775808,3.582321,-30.38765,--,-9223372036854775808,--,4.797
-9223372036854775808,3.596084,-30.38581,--,-9223372036854775808,--,7.887


In [12]:
#take new candels and skymatch to jwst to get jwst catalog
ra3=matched_candels['RA']
dec3=matched_candels['DEC']
catalog_candels = SkyCoord(ra=ra3*u.arcsec, dec=dec3*u.arcsec)#candels

In [14]:
idx_jwst, d2d_jwst, d3d_jwst = catalog_candels.match_to_catalog_sky(c) #jwst id returned
#d2d_as_jwst = (d2d_jwst/(u.degree))* u.arcsec

tst_jwst=[]
for i in range(len(ra3)):
    if d2d_jwst[i]< max_sep:
        tst_jwst.append(idx_jwst[i])
        
idx_jwst_new = list(set(tst_jwst))    #list of non-repeating jwst indexs

In [18]:
matched_jwst = jwst[idx_jwst_new]
matched_jwst.sort('ra')
matched_jwst

field,id,ra,dec,lupt_f1000w_aper0,lupterr_f1000w_aper0,lupt_f105w_aper0,lupterr_f105w_aper0,lupt_f115w-clear_aper0,lupterr_f115w-clear_aper0,lupt_f125w_aper0,lupterr_f125w_aper0,lupt_f1280w_aper0,lupterr_f1280w_aper0,lupt_f140w_aper0,lupterr_f140w_aper0,lupt_f1500w_aper0,lupterr_f1500w_aper0,lupt_f150w-clear_aper0,lupterr_f150w-clear_aper0,lupt_f160w_aper0,lupterr_f160w_aper0,lupt_f1800w_aper0,lupterr_f1800w_aper0,lupt_f182m-clear_aper0,lupterr_f182m-clear_aper0,lupt_f200w-clear_aper0,lupterr_f200w-clear_aper0,lupt_f2100w_aper0,lupterr_f2100w_aper0,lupt_f210m-clear_aper0,lupterr_f210m-clear_aper0,lupt_f275wu_aper0,lupterr_f275wu_aper0,lupt_f277w-clear_aper0,lupterr_f277w-clear_aper0,lupt_f356w-clear_aper0,lupterr_f356w-clear_aper0,lupt_f410m-clear_aper0,lupterr_f410m-clear_aper0,lupt_f435w_aper0,lupterr_f435w_aper0,lupt_f444w-clear_aper0,lupterr_f444w-clear_aper0,lupt_f560w_aper0,lupterr_f560w_aper0,lupt_f606w_aper0,lupterr_f606w_aper0,lupt_f606wu_aper0,lupterr_f606wu_aper0,lupt_f770w_aper0,lupterr_f770w_aper0,lupt_f814w_aper0,lupterr_f814w_aper0,lupt_f1000w_aper1,lupterr_f1000w_aper1,lupt_f105w_aper1,lupterr_f105w_aper1,lupt_f115w-clear_aper1,lupterr_f115w-clear_aper1,lupt_f125w_aper1,lupterr_f125w_aper1,lupt_f1280w_aper1,lupterr_f1280w_aper1,lupt_f140w_aper1,lupterr_f140w_aper1,lupt_f1500w_aper1,lupterr_f1500w_aper1,lupt_f150w-clear_aper1,lupterr_f150w-clear_aper1,lupt_f160w_aper1,lupterr_f160w_aper1,lupt_f1800w_aper1,lupterr_f1800w_aper1,lupt_f182m-clear_aper1,lupterr_f182m-clear_aper1,lupt_f200w-clear_aper1,lupterr_f200w-clear_aper1,lupt_f2100w_aper1,lupterr_f2100w_aper1,lupt_f210m-clear_aper1,lupterr_f210m-clear_aper1,lupt_f275wu_aper1,lupterr_f275wu_aper1,lupt_f277w-clear_aper1,lupterr_f277w-clear_aper1,lupt_f356w-clear_aper1,lupterr_f356w-clear_aper1,lupt_f410m-clear_aper1,lupterr_f410m-clear_aper1,lupt_f435w_aper1,lupterr_f435w_aper1,lupt_f444w-clear_aper1,lupterr_f444w-clear_aper1,lupt_f560w_aper1,lupterr_f560w_aper1,lupt_f606w_aper1,lupterr_f606w_aper1,lupt_f606wu_aper1,lupterr_f606wu_aper1,lupt_f770w_aper1,lupterr_f770w_aper1,lupt_f814w_aper1,lupterr_f814w_aper1,lupt_f1000w_aper2,lupterr_f1000w_aper2,lupt_f105w_aper2,lupterr_f105w_aper2,lupt_f115w-clear_aper2,lupterr_f115w-clear_aper2,lupt_f125w_aper2,lupterr_f125w_aper2,lupt_f1280w_aper2,lupterr_f1280w_aper2,lupt_f140w_aper2,lupterr_f140w_aper2,lupt_f1500w_aper2,lupterr_f1500w_aper2,lupt_f150w-clear_aper2,lupterr_f150w-clear_aper2,lupt_f160w_aper2,lupterr_f160w_aper2,lupt_f1800w_aper2,lupterr_f1800w_aper2,lupt_f182m-clear_aper2,lupterr_f182m-clear_aper2,lupt_f200w-clear_aper2,lupterr_f200w-clear_aper2,lupt_f2100w_aper2,lupterr_f2100w_aper2,lupt_f210m-clear_aper2,lupterr_f210m-clear_aper2,lupt_f275wu_aper2,lupterr_f275wu_aper2,lupt_f277w-clear_aper2,lupterr_f277w-clear_aper2,lupt_f356w-clear_aper2,lupterr_f356w-clear_aper2,lupt_f410m-clear_aper2,lupterr_f410m-clear_aper2,lupt_f435w_aper2,lupterr_f435w_aper2,lupt_f444w-clear_aper2,lupterr_f444w-clear_aper2,lupt_f560w_aper2,lupterr_f560w_aper2,lupt_f606w_aper2,lupterr_f606w_aper2,lupt_f606wu_aper2,lupterr_f606wu_aper2,lupt_f770w_aper2,lupterr_f770w_aper2,lupt_f814w_aper2,lupterr_f814w_aper2,lupt_f1000w_aper3,lupterr_f1000w_aper3,lupt_f105w_aper3,lupterr_f105w_aper3,lupt_f115w-clear_aper3,lupterr_f115w-clear_aper3,lupt_f125w_aper3,lupterr_f125w_aper3,lupt_f1280w_aper3,lupterr_f1280w_aper3,lupt_f140w_aper3,lupterr_f140w_aper3,lupt_f1500w_aper3,lupterr_f1500w_aper3,lupt_f150w-clear_aper3,lupterr_f150w-clear_aper3,lupt_f160w_aper3,lupterr_f160w_aper3,lupt_f1800w_aper3,lupterr_f1800w_aper3,lupt_f182m-clear_aper3,lupterr_f182m-clear_aper3,lupt_f200w-clear_aper3,lupterr_f200w-clear_aper3,lupt_f2100w_aper3,lupterr_f2100w_aper3,lupt_f210m-clear_aper3,lupterr_f210m-clear_aper3,lupt_f275wu_aper3,lupterr_f275wu_aper3,lupt_f277w-clear_aper3,lupterr_f277w-clear_aper3,lupt_f356w-clear_aper3,lupterr_f356w-clear_aper3,lupt_f410m-clear_aper3,lupterr_f410m-clear_aper3,lupt_f435w_aper3,lupterr_f435w_aper3,lupt_f444w-clear_aper3,lupterr_f

# Filter Choices for Training Catalogue


In [143]:
#PREMADE TRAINING CATALOGUE
train = Table.read('jwst_train_catalogue.fits',format = 'fits')
train

field,id_1,ra_1,dec_1,lupt_f1000w_aper0,lupterr_f1000w_aper0,lupt_f105w_aper0,lupterr_f105w_aper0,lupt_f115w-clear_aper0,lupterr_f115w-clear_aper0,lupt_f125w_aper0,lupterr_f125w_aper0,lupt_f1280w_aper0,lupterr_f1280w_aper0,lupt_f140w_aper0,lupterr_f140w_aper0,lupt_f1500w_aper0,lupterr_f1500w_aper0,lupt_f150w-clear_aper0,lupterr_f150w-clear_aper0,lupt_f160w_aper0,lupterr_f160w_aper0,lupt_f1800w_aper0,lupterr_f1800w_aper0,lupt_f182m-clear_aper0,lupterr_f182m-clear_aper0,lupt_f200w-clear_aper0,lupterr_f200w-clear_aper0,lupt_f2100w_aper0,lupterr_f2100w_aper0,lupt_f210m-clear_aper0,lupterr_f210m-clear_aper0,lupt_f275wu_aper0,lupterr_f275wu_aper0,lupt_f277w-clear_aper0,lupterr_f277w-clear_aper0,lupt_f356w-clear_aper0,lupterr_f356w-clear_aper0,lupt_f410m-clear_aper0,lupterr_f410m-clear_aper0,lupt_f435w_aper0,lupterr_f435w_aper0,lupt_f444w-clear_aper0,lupterr_f444w-clear_aper0,lupt_f560w_aper0,lupterr_f560w_aper0,lupt_f606w_aper0,lupterr_f606w_aper0,lupt_f606wu_aper0,lupterr_f606wu_aper0,lupt_f770w_aper0,lupterr_f770w_aper0,lupt_f814w_aper0,lupterr_f814w_aper0,lupt_f1000w_aper1,lupterr_f1000w_aper1,lupt_f105w_aper1,lupterr_f105w_aper1,lupt_f115w-clear_aper1,lupterr_f115w-clear_aper1,lupt_f125w_aper1,lupterr_f125w_aper1,lupt_f1280w_aper1,lupterr_f1280w_aper1,lupt_f140w_aper1,lupterr_f140w_aper1,lupt_f1500w_aper1,lupterr_f1500w_aper1,lupt_f150w-clear_aper1,lupterr_f150w-clear_aper1,lupt_f160w_aper1,lupterr_f160w_aper1,lupt_f1800w_aper1,lupterr_f1800w_aper1,lupt_f182m-clear_aper1,lupterr_f182m-clear_aper1,lupt_f200w-clear_aper1,lupterr_f200w-clear_aper1,lupt_f2100w_aper1,lupterr_f2100w_aper1,lupt_f210m-clear_aper1,lupterr_f210m-clear_aper1,lupt_f275wu_aper1,lupterr_f275wu_aper1,lupt_f277w-clear_aper1,lupterr_f277w-clear_aper1,lupt_f356w-clear_aper1,lupterr_f356w-clear_aper1,lupt_f410m-clear_aper1,lupterr_f410m-clear_aper1,lupt_f435w_aper1,lupterr_f435w_aper1,lupt_f444w-clear_aper1,lupterr_f444w-clear_aper1,lupt_f560w_aper1,lupterr_f560w_aper1,lupt_f606w_aper1,lupterr_f606w_aper1,lupt_f606wu_aper1,lupterr_f606wu_aper1,lupt_f770w_aper1,lupterr_f770w_aper1,lupt_f814w_aper1,lupterr_f814w_aper1,lupt_f1000w_aper2,lupterr_f1000w_aper2,lupt_f105w_aper2,lupterr_f105w_aper2,lupt_f115w-clear_aper2,lupterr_f115w-clear_aper2,lupt_f125w_aper2,lupterr_f125w_aper2,lupt_f1280w_aper2,lupterr_f1280w_aper2,lupt_f140w_aper2,lupterr_f140w_aper2,lupt_f1500w_aper2,lupterr_f1500w_aper2,lupt_f150w-clear_aper2,lupterr_f150w-clear_aper2,lupt_f160w_aper2,lupterr_f160w_aper2,lupt_f1800w_aper2,lupterr_f1800w_aper2,lupt_f182m-clear_aper2,lupterr_f182m-clear_aper2,lupt_f200w-clear_aper2,lupterr_f200w-clear_aper2,lupt_f2100w_aper2,lupterr_f2100w_aper2,lupt_f210m-clear_aper2,lupterr_f210m-clear_aper2,lupt_f275wu_aper2,lupterr_f275wu_aper2,lupt_f277w-clear_aper2,lupterr_f277w-clear_aper2,lupt_f356w-clear_aper2,lupterr_f356w-clear_aper2,lupt_f410m-clear_aper2,lupterr_f410m-clear_aper2,lupt_f435w_aper2,lupterr_f435w_aper2,lupt_f444w-clear_aper2,lupterr_f444w-clear_aper2,lupt_f560w_aper2,lupterr_f560w_aper2,lupt_f606w_aper2,lupterr_f606w_aper2,lupt_f606wu_aper2,lupterr_f606wu_aper2,lupt_f770w_aper2,lupterr_f770w_aper2,lupt_f814w_aper2,lupterr_f814w_aper2,lupt_f1000w_aper3,lupterr_f1000w_aper3,lupt_f105w_aper3,lupterr_f105w_aper3,lupt_f115w-clear_aper3,lupterr_f115w-clear_aper3,lupt_f125w_aper3,lupterr_f125w_aper3,lupt_f1280w_aper3,lupterr_f1280w_aper3,lupt_f140w_aper3,lupterr_f140w_aper3,lupt_f1500w_aper3,lupterr_f1500w_aper3,lupt_f150w-clear_aper3,lupterr_f150w-clear_aper3,lupt_f160w_aper3,lupterr_f160w_aper3,lupt_f1800w_aper3,lupterr_f1800w_aper3,lupt_f182m-clear_aper3,lupterr_f182m-clear_aper3,lupt_f200w-clear_aper3,lupterr_f200w-clear_aper3,lupt_f2100w_aper3,lupterr_f2100w_aper3,lupt_f210m-clear_aper3,lupterr_f210m-clear_aper3,lupt_f275wu_aper3,lupterr_f275wu_aper3,lupt_f277w-clear_aper3,lupterr_f277w-clear_aper3,lupt_f356w-clear_aper3,lupterr_f356w-clear_aper3,lupt_f410m-clear_aper3,lupterr_f410m-clear_aper3,lupt_f435w_aper3,lupterr_f435w_aper3,lupt_f444w-clear_aper3,lup

In [144]:
#Full catalogue filter names 
filters_1 = [col.split('_')[1] for col in train.colnames if col.endswith('_aper0')]
#print(len(filters_1))
x= int(184/4)+4
#lup_total['lupt_f250m-clear_aper0']
print(filters_1)

['f1000w', 'f1000w', 'f105w', 'f105w', 'f115w-clear', 'f115w-clear', 'f125w', 'f125w', 'f1280w', 'f1280w', 'f140w', 'f140w', 'f1500w', 'f1500w', 'f150w-clear', 'f150w-clear', 'f160w', 'f160w', 'f1800w', 'f1800w', 'f182m-clear', 'f182m-clear', 'f200w-clear', 'f200w-clear', 'f2100w', 'f2100w', 'f210m-clear', 'f210m-clear', 'f275wu', 'f275wu', 'f277w-clear', 'f277w-clear', 'f356w-clear', 'f356w-clear', 'f410m-clear', 'f410m-clear', 'f435w', 'f435w', 'f444w-clear', 'f444w-clear', 'f560w', 'f560w', 'f606w', 'f606w', 'f606wu', 'f606wu', 'f770w', 'f770w', 'f814w', 'f814w', 'f090w-clear', 'f090w-clear', 'f335m-clear', 'f335m-clear', 'f350lpu', 'f350lpu', 'f475w', 'f475w', 'f775w', 'f775w', 'f850lp', 'f850lp', 'f110w', 'f110w', 'f336wu', 'f336wu', 'f850lpu', 'f850lpu', 'clearp-f356w', 'clearp-f356w', 'clearp-f444w', 'clearp-f444w', 'f115wn-clear', 'f115wn-clear', 'f150wn-clear', 'f150wn-clear', 'f200wn-clear', 'f200wn-clear', 'f225wu', 'f225wu', 'f250m-clear', 'f250m-clear', 'f300m-clear', 'f30

In [211]:
#### FILTER CHOICES
c1 =['f435w','f105w','f277w-clear','f356w-clear','f444w-clear']
c2 =['f606w','f150w-clear','f210m-clear','f356w-clear','f444w-clear']
c3 =['f814w','f140w','f277w-clear','f410m-clear','f444w-clear']
#need to fix c4 and c5
c4 =['f775w','f160w','f200w-clear','f356w-clear','f410m-clear']
c5 =['f606w','f110w','f200w-clear','f356w-clear','f444w-clear']


In [225]:
######change this to make new catalogue of filters
aper=0
choice=c1
lup_train = train['field','id_1','ra_1','dec_1','z_spec']
lup_train[f'lupt_{choice[0]}']=train[f'lupt_{choice[0]}_aper{aper}']
lup_train[f'lupterr_{choice[0]}']=train[f'lupterr_{choice[0]}_aper{aper}']
lup_train[f'lupt_{choice[1]}']=train[f'lupt_{choice[1]}_aper{aper}']
lup_train[f'lupterr_{choice[1]}']=train[f'lupterr_{choice[1]}_aper{aper}']
lup_train[f'lupt_{choice[2]}']=train[f'lupt_{choice[2]}_aper{aper}']
lup_train[f'lupterr_{choice[2]}']=train[f'lupterr_{choice[2]}_aper{aper}']
lup_train[f'lupt_{choice[3]}']=train[f'lupt_{choice[3]}_aper{aper}']
lup_train[f'lupterr_{choice[3]}']=train[f'lupterr_{choice[3]}_aper{aper}']
lup_train[f'lupt_{choice[4]}']=train[f'lupt_{choice[4]}_aper{aper}']
lup_train[f'lupterr_{choice[4]}']=train[f'lupterr_{choice[4]}_aper{aper}']

In [226]:
lup_train

field,id_1,ra_1,dec_1,z_spec,lupt_f435w,lupterr_f435w,lupt_f105w,lupterr_f105w,lupt_f277w-clear,lupterr_f277w-clear,lupt_f356w-clear,lupterr_f356w-clear,lupt_f444w-clear,lupterr_f444w-clear
bytes13,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ceers,789,215.15422053802376,52.95581829662729,4.997,30.54264947281063,0.3511188054513726,--,--,--,--,26.136482866132383,0.043272126037807855,--,--
ceers,5587,214.96000650701987,52.83116263905752,4.897,29.941254849175753,0.5879821889951125,--,--,26.29346092851237,0.017913357202100192,26.53782392779815,0.018479351549360225,26.569316560669414,0.027425939965754813
ceers,6068,214.84308865874448,52.74788317482039,7.509,29.432030130683614,0.5800563213930446,28.83929184236302,0.959315035431026,27.803168931744715,0.1115439323129621,27.913404656612578,0.13519856466238045,27.479142172392997,0.13782790379332446
ceers,7389,215.14530062350434,52.967285494500544,4.48,31.00436325965363,0.2913999654731623,--,--,26.23274491338961,0.015875044157524918,26.41593755416214,0.015392950486609826,26.656393525386978,0.03654841317026106
ceers,8155,214.95670277336635,52.83377515425651,5.736,29.66841821117542,0.5587122977384609,--,--,28.39219158932994,0.10760109421195524,27.87283799064398,0.05618533103217969,28.355131068247115,0.13247934146476611
ceers,8456,215.0353875926934,52.89067150664201,8.679,29.48214980984941,0.5639223963372081,28.38139119576731,0.17165019505744594,25.683577484181985,0.007314253614478422,25.45285249335856,0.006193299488997425,24.554128178559036,0.004178595334847659
ceers,8464,215.1520355562678,52.97404432091189,4.482,29.620498534828254,0.36779936133681457,--,--,24.930449893155338,0.006022626536135842,24.78461870915859,0.004762971084677135,24.857357104962947,0.009010314078062544
ceers,8992,214.8205024300106,52.73714320681697,6.272,--,--,--,--,28.434582057466095,0.1314315655893294,27.581856929302173,0.05093275273232454,28.260210276598123,0.1643612145187621
ceers,9134,214.82837104152972,52.742942723665365,4.493,30.956617679085824,0.5175584644700718,--,--,27.650579631089204,0.059607284535686006,27.820618076861738,0.061759849557884015,28.108972455225455,0.13397633475740922


In [ ]:
#filts=['f115w','f200w','f277w','f356w','f444w']

In [227]:
f1=[]
for i in range(len(lup_train)):
    f1.append(np.ma.is_masked(lup_train[f'lupt_{choice[0]}'][i]))
f1_inv=np.array(np.invert(f1))
f1_cat =lup_train[f1_inv*(np.ma.is_masked(lup_train[f'lupt_{choice[0]}']) == True)] 

In [228]:
f1_cat

field,id_1,ra_1,dec_1,z_spec,lupt_f435w,lupterr_f435w,lupt_f105w,lupterr_f105w,lupt_f277w-clear,lupterr_f277w-clear,lupt_f356w-clear,lupterr_f356w-clear,lupt_f444w-clear,lupterr_f444w-clear
bytes13,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ceers,789,215.15422053802376,52.95581829662729,4.997,30.54264947281063,0.3511188054513726,--,--,--,--,26.136482866132383,0.043272126037807855,--,--
ceers,5587,214.96000650701987,52.83116263905752,4.897,29.941254849175753,0.5879821889951125,--,--,26.29346092851237,0.017913357202100192,26.53782392779815,0.018479351549360225,26.569316560669414,0.027425939965754813
ceers,6068,214.84308865874448,52.74788317482039,7.509,29.432030130683614,0.5800563213930446,28.83929184236302,0.959315035431026,27.803168931744715,0.1115439323129621,27.913404656612578,0.13519856466238045,27.479142172392997,0.13782790379332446
ceers,7389,215.14530062350434,52.967285494500544,4.48,31.00436325965363,0.2913999654731623,--,--,26.23274491338961,0.015875044157524918,26.41593755416214,0.015392950486609826,26.656393525386978,0.03654841317026106
ceers,8155,214.95670277336635,52.83377515425651,5.736,29.66841821117542,0.5587122977384609,--,--,28.39219158932994,0.10760109421195524,27.87283799064398,0.05618533103217969,28.355131068247115,0.13247934146476611
ceers,8456,215.0353875926934,52.89067150664201,8.679,29.48214980984941,0.5639223963372081,28.38139119576731,0.17165019505744594,25.683577484181985,0.007314253614478422,25.45285249335856,0.006193299488997425,24.554128178559036,0.004178595334847659
ceers,8464,215.1520355562678,52.97404432091189,4.482,29.620498534828254,0.36779936133681457,--,--,24.930449893155338,0.006022626536135842,24.78461870915859,0.004762971084677135,24.857357104962947,0.009010314078062544
ceers,9134,214.82837104152972,52.742942723665365,4.493,30.956617679085824,0.5175584644700718,--,--,27.650579631089204,0.059607284535686006,27.820618076861738,0.061759849557884015,28.108972455225455,0.13397633475740922
ceers,10874,214.96128013286693,52.84235079261266,8.638,30.253540235991355,0.6141729789250264,--,--,28.5965242369499,0.12380479181541451,28.39820206532098,0.09501750953841197,27.84294761047686,0.07942371941313843


In [234]:

f2=[]
for i in range(len(f1_cat)):
    f2.append(np.ma.is_masked(f1_cat[f'lupt_{choice[1]}'][i]))
f2_inv=np.array(np.invert(f2))
f2_cat =f1_cat[f2_inv*(np.ma.is_masked(f1_cat[f'lupt_{choice[1]}']) ==True)] 

In [235]:
f2_cat

field,id_1,ra_1,dec_1,z_spec,lupt_f435w,lupterr_f435w,lupt_f105w,lupterr_f105w,lupt_f277w-clear,lupterr_f277w-clear,lupt_f356w-clear,lupterr_f356w-clear,lupt_f444w-clear,lupterr_f444w-clear
bytes13,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ceers,6068,214.84308865874448,52.74788317482039,7.509,29.432030130683614,0.5800563213930446,28.83929184236302,0.959315035431026,27.803168931744715,0.1115439323129621,27.913404656612578,0.13519856466238045,27.479142172392997,0.13782790379332446
ceers,8456,215.0353875926934,52.89067150664201,8.679,29.48214980984941,0.5639223963372081,28.38139119576731,0.17165019505744594,25.683577484181985,0.007314253614478422,25.45285249335856,0.006193299488997425,24.554128178559036,0.004178595334847659
ceers,12671,214.83120294678287,52.75178168803372,4.995,28.9739807670179,0.46834135051934483,30.02850145839513,0.7461217417732501,28.008113273979028,0.07623575155330445,28.562602663760703,0.11650231312863897,28.59253920957155,0.19426657584734686
ceers,16547,215.03054186857705,52.90259367528189,5.007,30.200270786478516,0.638997671826721,27.287167090133334,0.07799374494579508,26.872435806364408,0.01638310148562873,27.190937589695956,0.02002634918121507,27.210988742643124,0.030434562858984052
ceers,16991,215.15088639972902,52.989554617408544,7.752,30.81606490344799,0.3409653133043009,27.787403446827376,0.12178060554749814,--,--,27.139249834052183,0.10904054544662965,--,--
ceers,17316,214.8120537351543,52.74674399702545,7.477,29.22690297390089,0.513982605860812,28.188982183599034,0.6667582559722047,27.470612165238585,0.04731585387589971,27.42389908498038,0.04100904870223872,26.80286603720184,0.038803905348054744
ceers,18083,214.821419325401,52.75483654660109,4.307,30.16170033079362,0.703544392562639,27.644213628672823,0.5411778311947344,26.834762389864192,0.026782448216464708,27.364494371237363,0.039773459297327916,27.968358898349383,0.11381130560776387
ceers,20028,215.0909456713268,52.951531430267416,5.504,30.14224800006085,0.3909827594341157,27.29696269945327,0.18518145252606882,26.843209318011922,0.02437842565450855,26.391891585598035,0.015445998614647133,26.708544449088276,0.03574438010789914
ceers,20595,215.02691380412827,52.907204336730004,4.307,29.621621260142327,0.5622775794256634,27.034937708791503,0.0635355950301793,26.184232434739748,0.009844374826585966,26.38083078070472,0.010976009052930318,26.621537496765974,0.01846571654685996


In [236]:

f3=[]
for i in range(len(f2_cat)):
    f3.append(np.ma.is_masked(f2_cat[f'lupt_{choice[2]}'][i]))
f3_inv=np.array(np.invert(f3))
f3_cat =f2_cat[f3_inv*(np.ma.is_masked(f2_cat[f'lupt_{choice[2]}']) == True)] 

In [237]:
f3_cat

field,id_1,ra_1,dec_1,z_spec,lupt_f435w,lupterr_f435w,lupt_f105w,lupterr_f105w,lupt_f277w-clear,lupterr_f277w-clear,lupt_f356w-clear,lupterr_f356w-clear,lupt_f444w-clear,lupterr_f444w-clear
bytes13,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ceers,6068,214.84308865874448,52.74788317482039,7.509,29.432030130683614,0.5800563213930446,28.83929184236302,0.959315035431026,27.803168931744715,0.1115439323129621,27.913404656612578,0.13519856466238045,27.479142172392997,0.13782790379332446
ceers,8456,215.0353875926934,52.89067150664201,8.679,29.48214980984941,0.5639223963372081,28.38139119576731,0.17165019505744594,25.683577484181985,0.007314253614478422,25.45285249335856,0.006193299488997425,24.554128178559036,0.004178595334847659
ceers,12671,214.83120294678287,52.75178168803372,4.995,28.9739807670179,0.46834135051934483,30.02850145839513,0.7461217417732501,28.008113273979028,0.07623575155330445,28.562602663760703,0.11650231312863897,28.59253920957155,0.19426657584734686
ceers,16547,215.03054186857705,52.90259367528189,5.007,30.200270786478516,0.638997671826721,27.287167090133334,0.07799374494579508,26.872435806364408,0.01638310148562873,27.190937589695956,0.02002634918121507,27.210988742643124,0.030434562858984052
ceers,17316,214.8120537351543,52.74674399702545,7.477,29.22690297390089,0.513982605860812,28.188982183599034,0.6667582559722047,27.470612165238585,0.04731585387589971,27.42389908498038,0.04100904870223872,26.80286603720184,0.038803905348054744
ceers,18083,214.821419325401,52.75483654660109,4.307,30.16170033079362,0.703544392562639,27.644213628672823,0.5411778311947344,26.834762389864192,0.026782448216464708,27.364494371237363,0.039773459297327916,27.968358898349383,0.11381130560776387
ceers,20028,215.0909456713268,52.951531430267416,5.504,30.14224800006085,0.3909827594341157,27.29696269945327,0.18518145252606882,26.843209318011922,0.02437842565450855,26.391891585598035,0.015445998614647133,26.708544449088276,0.03574438010789914
ceers,20595,215.02691380412827,52.907204336730004,4.307,29.621621260142327,0.5622775794256634,27.034937708791503,0.0635355950301793,26.184232434739748,0.009844374826585966,26.38083078070472,0.010976009052930318,26.621537496765974,0.01846571654685996
ceers,21668,214.81262715994268,52.75484607452653,5.427,30.091445228652773,0.8079811391844782,27.65475736574522,0.48662171162695145,27.49940693193139,0.05172527293420636,27.2254546791845,0.0362900315018761,27.576923726543313,0.08127279446108882


In [238]:

f4=[]
for i in range(len(f3_cat)):
    f4.append(np.ma.is_masked(f3_cat[f'lupt_{choice[3]}'][i]))
f4_inv=np.array(np.invert(f4))
f4_cat =f3_cat[f4_inv*(np.ma.is_masked(f3_cat[f'lupt_{choice[3]}']) == True)] #false idk why but works 


In [239]:
f4_cat

field,id_1,ra_1,dec_1,z_spec,lupt_f435w,lupterr_f435w,lupt_f105w,lupterr_f105w,lupt_f277w-clear,lupterr_f277w-clear,lupt_f356w-clear,lupterr_f356w-clear,lupt_f444w-clear,lupterr_f444w-clear
bytes13,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ceers,6068,214.84308865874448,52.74788317482039,7.509,29.432030130683614,0.5800563213930446,28.83929184236302,0.959315035431026,27.803168931744715,0.1115439323129621,27.913404656612578,0.13519856466238045,27.479142172392997,0.13782790379332446
ceers,8456,215.0353875926934,52.89067150664201,8.679,29.48214980984941,0.5639223963372081,28.38139119576731,0.17165019505744594,25.683577484181985,0.007314253614478422,25.45285249335856,0.006193299488997425,24.554128178559036,0.004178595334847659
ceers,12671,214.83120294678287,52.75178168803372,4.995,28.9739807670179,0.46834135051934483,30.02850145839513,0.7461217417732501,28.008113273979028,0.07623575155330445,28.562602663760703,0.11650231312863897,28.59253920957155,0.19426657584734686
ceers,16547,215.03054186857705,52.90259367528189,5.007,30.200270786478516,0.638997671826721,27.287167090133334,0.07799374494579508,26.872435806364408,0.01638310148562873,27.190937589695956,0.02002634918121507,27.210988742643124,0.030434562858984052
ceers,17316,214.8120537351543,52.74674399702545,7.477,29.22690297390089,0.513982605860812,28.188982183599034,0.6667582559722047,27.470612165238585,0.04731585387589971,27.42389908498038,0.04100904870223872,26.80286603720184,0.038803905348054744
ceers,18083,214.821419325401,52.75483654660109,4.307,30.16170033079362,0.703544392562639,27.644213628672823,0.5411778311947344,26.834762389864192,0.026782448216464708,27.364494371237363,0.039773459297327916,27.968358898349383,0.11381130560776387
ceers,20028,215.0909456713268,52.951531430267416,5.504,30.14224800006085,0.3909827594341157,27.29696269945327,0.18518145252606882,26.843209318011922,0.02437842565450855,26.391891585598035,0.015445998614647133,26.708544449088276,0.03574438010789914
ceers,20595,215.02691380412827,52.907204336730004,4.307,29.621621260142327,0.5622775794256634,27.034937708791503,0.0635355950301793,26.184232434739748,0.009844374826585966,26.38083078070472,0.010976009052930318,26.621537496765974,0.01846571654685996
ceers,21668,214.81262715994268,52.75484607452653,5.427,30.091445228652773,0.8079811391844782,27.65475736574522,0.48662171162695145,27.49940693193139,0.05172527293420636,27.2254546791845,0.0362900315018761,27.576923726543313,0.08127279446108882


In [240]:
f5=[]
for i in range(len(f4_cat)):
    f5.append(np.ma.is_masked(f4_cat[f'lupt_{choice[4]}'][i]))
f5_inv=np.array(np.invert(f5))
f5_cat =f4_cat[f5_inv*(np.ma.is_masked(f4_cat[f'lupt_{choice[4]}']) == True)] 

In [241]:
f5_cat

field,id_1,ra_1,dec_1,z_spec,lupt_f435w,lupterr_f435w,lupt_f105w,lupterr_f105w,lupt_f277w-clear,lupterr_f277w-clear,lupt_f356w-clear,lupterr_f356w-clear,lupt_f444w-clear,lupterr_f444w-clear
bytes13,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ceers,6068,214.84308865874448,52.74788317482039,7.509,29.432030130683614,0.5800563213930446,28.83929184236302,0.959315035431026,27.803168931744715,0.1115439323129621,27.913404656612578,0.13519856466238045,27.479142172392997,0.13782790379332446
ceers,8456,215.0353875926934,52.89067150664201,8.679,29.48214980984941,0.5639223963372081,28.38139119576731,0.17165019505744594,25.683577484181985,0.007314253614478422,25.45285249335856,0.006193299488997425,24.554128178559036,0.004178595334847659
ceers,12671,214.83120294678287,52.75178168803372,4.995,28.9739807670179,0.46834135051934483,30.02850145839513,0.7461217417732501,28.008113273979028,0.07623575155330445,28.562602663760703,0.11650231312863897,28.59253920957155,0.19426657584734686
ceers,16547,215.03054186857705,52.90259367528189,5.007,30.200270786478516,0.638997671826721,27.287167090133334,0.07799374494579508,26.872435806364408,0.01638310148562873,27.190937589695956,0.02002634918121507,27.210988742643124,0.030434562858984052
ceers,17316,214.8120537351543,52.74674399702545,7.477,29.22690297390089,0.513982605860812,28.188982183599034,0.6667582559722047,27.470612165238585,0.04731585387589971,27.42389908498038,0.04100904870223872,26.80286603720184,0.038803905348054744
ceers,18083,214.821419325401,52.75483654660109,4.307,30.16170033079362,0.703544392562639,27.644213628672823,0.5411778311947344,26.834762389864192,0.026782448216464708,27.364494371237363,0.039773459297327916,27.968358898349383,0.11381130560776387
ceers,20028,215.0909456713268,52.951531430267416,5.504,30.14224800006085,0.3909827594341157,27.29696269945327,0.18518145252606882,26.843209318011922,0.02437842565450855,26.391891585598035,0.015445998614647133,26.708544449088276,0.03574438010789914
ceers,20595,215.02691380412827,52.907204336730004,4.307,29.621621260142327,0.5622775794256634,27.034937708791503,0.0635355950301793,26.184232434739748,0.009844374826585966,26.38083078070472,0.010976009052930318,26.621537496765974,0.01846571654685996
ceers,21668,214.81262715994268,52.75484607452653,5.427,30.091445228652773,0.8079811391844782,27.65475736574522,0.48662171162695145,27.49940693193139,0.05172527293420636,27.2254546791845,0.0362900315018761,27.576923726543313,0.08127279446108882


In [242]:
f5_cat.write('MPhys/catalogues/filter_choice_1.fits',format = 'fits',overwrite=True)

In [4]:
fit = Table.read('MPhys/catalogues/filter_choice_2.fits',format = 'fits')
fit.write('MPhys/catalogues/filter_choice_2.cat',format = 'ascii.commented_header',overwrite=True)

In [243]:
f5_cat.write('MPhys/catalogues/filter_choice_1.cat',format = 'ascii.commented_header',overwrite=True)

In [5]:
t_full =Table.read('MPhys/catalogues/filter_choice_2.cat',format = 'ascii.commented_header')
train_sample = int(len(t_full)*0.8)

In [6]:
t = t_full[0:train_sample]
p = t_full[train_sample:-1]

In [7]:
t.write('MPhys/catalogues/filter_choice_2_TRAIN.cat',format = 'ascii.commented_header')
p.write('MPhys/catalogues/filter_choice_2_PRED.cat',format = 'ascii.commented_header')

In [21]:
#IGNORE EVERYTHING ELSE - MOVING TO ANOTHER NOTEBOOK 

6277

In [ ]:
fit= Table.read('MPhys/jwst_parameter_files/jwst_nans_removed_lupts_specz_TRAIN.fits', format = 'fits')
train = fit['id_1','ra_1','dec_1','lupt_f115w','lupterr_f115w','lupt_f200w','lupterr_f200w','lupt_f277w','lupterr_f277w','lupt_f356w','lupterr_f356w','lupt_f444w','lupterr_f444w','z_spec']
train.write('MPhys/gpz_files/jwst_TRAIN_2.cat',format='ascii.commented_header',overwrite=True)

In [ ]:
train.sort('id_1')

In [ ]:
train_split=train[0:int(0.8*len(train))]
pred_split=train[int(0.8*len(train)):-1]

In [ ]:
train_split.write('MPhys/gpz_files/TRAIN_80_split.cat', format ='ascii.commented_header', overwrite=True)
pred_split.write('MPhys/gpz_files/PRED_20_split.cat', format ='ascii.commented_header', overwrite=True)

In [ ]:
pred_split

In [ ]:
ceers_new.write('ceers_tst.fits',format='fits',overwrite=True)

In [ ]:
filt='f115w-clear_flux_aper_1'
cat_ceers = ceers[f'{filt}','flux_auto','flux_aper_1']
cat_gds= gds[f'{filt}','flux_auto','flux_aper_1']
cat_gdn = gdn[f'{filt}','flux_auto','flux_aper_1']
cat_ngdeep = ngdeep[f'{filt}','flux_auto','flux_aper_1']
cat_cosmos_west = cosmos_west[f'{filt}','flux_auto','flux_aper_1']
cat_cosmos_east = cosmos_east[f'{filt}','flux_auto','flux_aper_1']
cat_udss = udss[f'{filt}','flux_auto','flux_aper_1']
cat_udsn = udsn[f'{filt}','flux_auto','flux_aper_1']


    


In [ ]:
flux_ceers=[]
mag_ceers=[]

flux_gds=[]
mag_gds=[]

flux_gdn=[]
#mag_gdn=[]

flux_ngdeep=[]
mag_ngdeep=[]

flux_cosmos_west=[]
mag_cosmos_west=[]

flux_udss=[]
mag_udss=[]

flux_udsn=[]
#mag_udsn=[]

flux_cosmos_east=[]
#mag_cosmos_east=[]

for j in range(len(cat_ceers)):
    flux_ceers.append((cat_ceers[f'{filt}'][j] * cat_ceers['flux_auto'][j]) /cat_ceers['flux_aper_1'][j])
    mag_ceers.append(23.9-(2.5*np.log10(flux_ceers[j])))
for j in range(len(cat_gds)):
    flux_gds.append((cat_gds[f'{filt}'][j] * cat_gds['flux_auto'][j]) /cat_gds['flux_aper_1'][j])
    mag_gds.append(23.9-(2.5*np.log10(flux_gds[j])))
for j in range(len(cat_gdn)):
    flux_gdn.append((cat_gdn[f'{filt}'][j] * cat_gdn['flux_auto'][j]) /cat_gdn['flux_aper_1'][j])
    mag_gds.append(23.9-(2.5*np.log10(flux_gdn[j])))
for j in range(len(cat_cosmos_west)):
    flux_cosmos_west.append((cat_cosmos_west[f'{filt}'][j] * cat_cosmos_west['flux_auto'][j]) /cat_cosmos_west['flux_aper_1'][j])
    mag_cosmos_west.append(23.9-(2.5*np.log10(flux_cosmos_west[j])))
for j in range(len(cat_cosmos_east)):
    flux_cosmos_east.append((cat_cosmos_east[f'{filt}'][j] * cat_cosmos_east['flux_auto'][j]) /cat_cosmos_east['flux_aper_1'][j])
    mag_cosmos_west.append(23.9-(2.5*np.log10(flux_cosmos_east[j])))
for j in range(len(cat_udss)):
    flux_udss.append((cat_udss[f'{filt}'][j] * cat_udss['flux_auto'][j]) /cat_udss['flux_aper_1'][j])
    mag_udss.append(23.9-(2.5*np.log10(flux_udss[j])))
for j in range(len(cat_udsn)):
    flux_udsn.append((cat_udsn[f'{filt}'][j] * cat_udsn['flux_auto'][j]) /cat_udsn['flux_aper_1'][j])
    mag_udss.append(23.9-(2.5*np.log10(flux_udsn[j])))
for j in range(len(cat_ngdeep)):
    flux_ngdeep.append((cat_ngdeep[f'{filt}'][j] * cat_ngdeep['flux_auto'][j]) /cat_ngdeep['flux_aper_1'][j])
    mag_ngdeep.append(23.9-(2.5*np.log10(flux_ngdeep[j])))


In [ ]:
plt.hist(mag_ceers,histtype='step',bins=100,label='CEERS')
plt.hist(mag_gds,histtype='step',bins=100,label='GDS')
plt.hist(mag_cosmos_west,histtype='step',bins=100,label='PRIMER COSMOS')
plt.hist(mag_udss,histtype='step',bins=100,label='UDS')
plt.hist(mag_ngdeep,histtype='step',bins=100,label='NGDeep')
plt.xlabel('AB Magnitudes')
plt.title(f'AB Magnitudes for {filt}')
plt.legend()
plt.show()
